# Propagation in Naturalis

To propagate orbits in Naturalis, use the following imports from the `naturalis.dynamics.propagator` module.


In [1]:
from naturalis.dynamics.propagator import OrbitalPropagator, ForceModel, PropagationMethod

The propagator takes in a list of `ForceModels` and a `PropagationMethod`. Here we will only use a single forces, the central two-body force, and we will use the `RK45` method.

In [2]:
propagator = OrbitalPropagator([ForceModel.TWO_BODY], PropagationMethod.RK45)

Next, we import Earth's specific gravitation parameters `MU` from `naturalis.constants` and the `OrbitalState` class from `naturalis.dynamics.orbit`. Since `position` and `velocity` are represented with `numpy` arrays, we will also need to import that. We will define an initial state that marks the start of propagation, which represents a circular Low-Earth orbit. 

In [3]:
from naturalis.dynamics.orbit import OrbitalState
from naturalis.constants import MU

import numpy as np

initial_state = OrbitalState(
    mu=MU,
    time=0,
    position=np.array([7000.0, 0.0, 0.0]),
    velocity=np.array([0.0, 7.546, 0.0])
)

Finally, we can propagate the state forward with the `propagate_by_time()` function. This propagates by a relative time (initial state time plus given time), but we could also plot to an absolute time with `propagate_to_time()`.

In [4]:
final_state = propagator.propagate_by_time(
    initial_state,
    1000.0
)

We can print out the representation to see that we have indeed propagated the state.

In [5]:
final_state

OrbitalState(mu=398600.4418, time=np.float64(1000.0), position=array([3311.58038824, 6167.06623909,    0.        ]), velocity=array([-6.64824078,  3.56985407,  0.        ]))

To view our states graphically, we can import `plot_orbital_states()` from `naturalis.plotting`. We must pass in a list of states and a `plotly.graph_objects.Figure` object. The latter is done so you can stack graphs on a single figure. Remember to use the `show()` function in order to see your plots.

In [6]:
from naturalis.plotting import plot_orbital_states
import plotly.graph_objects as go

fig = go.Figure()
plot_orbital_states([initial_state, final_state], fig)
fig.show()

Now we can clearly see that state has been propagated! Hover over each state to see more information about it.